In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
import openpyxl

In [11]:
df= pd.read_csv("datasets/CC GENERAL.csv", sep="," , decimal=".")
df.fillna(value=0, inplace=True) # Lleno los lugares vacíos con cero
credit_card= df.iloc[:,1:].values # Transformo los datos para la clusterización
normalizarDatos= StandardScaler() # Normalizar Datos
credit_card=normalizarDatos.fit_transform(credit_card)
print(credit_card)

[[-0.73198937 -0.24943448 -0.42489974 ... -0.2973097  -0.52555097
   0.36067954]
 [ 0.78696085  0.13432467 -0.46955188 ...  0.10204243  0.2342269
   0.36067954]
 [ 0.44713513  0.51808382 -0.10766823 ... -0.08848934 -0.52555097
   0.36067954]
 ...
 [-0.7403981  -0.18547673 -0.40196519 ... -0.32175099  0.32919999
  -4.12276757]
 [-0.74517423 -0.18547673 -0.46955188 ... -0.33316552  0.32919999
  -4.12276757]
 [-0.57257511 -0.88903307  0.04214581 ... -0.31923775 -0.52555097
  -4.12276757]]


In [4]:
# Vemos el número de cluster adecuados. Este es donde ocurre el número optimo de cluster,
wcssCC= [] # WCSS (Within-Cluster Sum of Squares) se refiere a la Suma de Cuadrados Dentro del Clúster. Es una métrica utilizada para evaluar la calidad de la agrupación en algoritmos como K-Means
for i in range(1, 11):
    kmeansCC=KMeans(n_clusters=i, init="k-means++", max_iter=300, n_init=10, random_state=0)
    kmeansCC.fit(credit_card)
    wcssCC.append(kmeansCC.inertia_)

# Imprimimos cada uno de los valores de los cluster para ver como salió
for i in range(len(wcssCC)):
    print(f'Cluster{i+1}:{wcssCC[i]}')

Cluster1:152150.0
Cluster2:127784.00581142175
Cluster3:111967.3727886735
Cluster4:99055.62855595461
Cluster5:91484.92413050328
Cluster6:84809.97765569761
Cluster7:79708.86018954834
Cluster8:74450.34459969406
Cluster9:69810.35879797644
Cluster10:66457.80442091817


In [5]:
# Graficamos nuestro diagrama del codo para ver donde quedo la menor variación de nuestra curva y reconocer el número óptimo del clusters
grafico=px.line(x=range(1,11),y=wcssCC)
grafico.update_xaxes(title_text="Número de Clusters")
grafico.update_yaxes(title_text="Valor de Wcss")
grafico.update_layout(title_text="Gráfico de Cotovelo", title_x=0.5)
grafico.show()

In [7]:
# Creamos una columna que nos da el cluster de cada cliente en el dataframe
kmeansCCFinal= KMeans(n_clusters=4, random_state=0)
y_kmeansCC=kmeansCCFinal.fit_predict(credit_card) # Aquí guardamnos a que cluster pertenece cada cliente
centroides= kmeansCCFinal.cluster_centers_
df["Cluster"]= y_kmeansCC

In [8]:
# Análisis vectorial de componentes principales
from sklearn.decomposition import PCA # análisis vectorial de componentes principales
pca=PCA(n_components=2) # solo 2 componenetes ya que solo queremos una grafica de 2D
credit_card_2d=pca.fit_transform(credit_card)
df["PCA1"]=credit_card_2d[:,0]
df["PCA2"]=credit_card_2d[:,1]
df_cluster= df[['CUST_ID', 'PCA1', 'PCA2', 'Cluster']]
df_cluster.head(10)


,CUST_ID,PCA1,PCA2,Cluster
0,C10001,-1.684378,-1.069779,1
1,C10002,-1.131680,2.510696,2
2,C10003,0.969303,-0.383534,3
3,C10004,-0.897010,-0.017484,1
4,C10005,-1.600270,-0.681000,1
5,C10006,0.253870,-0.774467,3
6,C10007,6.348683,-0.711145,0
7,C10008,0.275571,-1.286634,3
8,C10009,-0.442325,-0.458098,1
9,C10010,-0.564502,-0.366879,1


In [9]:
# Grafico de mis segmentaciones por tipo de cliente.
fig=px.scatter(df,x="PCA1", y="PCA2", color="Cluster", title="Cluster Visualization")
fig.show()

In [ ]:
# Guardarlo en un excel
out_path= "datasets/CC_GENERAL_WITH_Clusters.xlsx"
df.to_excel(out_path, index=False)